In [1]:
!pip install FlagEmbedding peft

     ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
     ------ ------------------------------ 30.7/163.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 163.8/163.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/374.8 kB ? eta -:--:--
   ---------------------------------------- 374.8/374.8 kB 7.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/342.1 kB ? eta -:--:--
   --------------------------------------- 342.1/342.1 kB 10.4 MB/s eta 0:00:00
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


기계 독해 데이터셋을 다운로드합니다.

In [3]:
!wget https://github.com/KLUE-benchmark/KLUE/raw/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [8]:
import requests

url = "https://github.com/KLUE-benchmark/KLUE/raw/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json"
response = requests.get(url)

# 파일 저장
with open("klue-mrc-v1.1_train.json", "wb") as f:
    f.write(response.content)

print("파일 다운로드 완료")

파일 다운로드 완료


In [9]:
import os

file_path = "klue-mrc-v1.1_train.json"
print(f"파일 크기: {os.path.getsize(file_path)} 바이트")

파일 크기: 47952737 바이트


In [5]:
import json
import pandas as pd
from tqdm import tqdm

In [10]:
with open('klue-mrc-v1.1_train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 필요한 정보 추출
records = []
for article in data['data']:
    title = article['title']
    news_category = article.get('news_category', 'N/A')
    source = article.get('source', 'N/A')

    for paragraph in article['paragraphs']:
        context = paragraph['context']

        for qa in paragraph['qas']:
            question = qa['question']
            question_type = qa['question_type']
            is_impossible = qa['is_impossible']

            for answer in qa['answers']:
                answer_text = answer['text']
                answer_start = answer['answer_start']

                records.append({
                    'title': title,
                    'news_category': news_category,
                    'source': source,
                    'context': context,
                    'question': question,
                    'question_type': question_type,
                    'is_impossible': is_impossible,
                    'answer_text': answer_text,
                    'answer_start': answer_start
                })

# DataFrame으로 변환
df = pd.DataFrame(records)

In [11]:
df = df.drop_duplicates(subset='question')

In [12]:
df = df.drop_duplicates(subset='context')

In [13]:
df[['context', 'question', 'answer_text']]

,context,question,answer_text
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,한 달가량
2,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?,삼보테크놀로지
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,와쿠이 히데아키
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,‘교동반점 짬뽕’
8,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,환경오염이 심한 지역
...,...,...,...
17656,“오너 경영은 중견기업이 글로벌 전문기업이나 대기업으로 성장하는 밑거름이 될 것입니...,이동기 학회장은 오너 경영을 무엇과 비교했는가?,전문 경영인 체제
17657,홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?,롯데백화점 잠실점
17658,유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?,‘혹성탈출: 반격의 서막’
17660,ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?,8시 10분


In [14]:
df['context'].loc[0]

'올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.'

In [15]:
df['question'].loc[0]

'북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?'

In [16]:
df['answer_text'].loc[0]

'한 달가량'

In [18]:
from FlagEmbedding import BGEM3FlagModel
import numpy as np
from tqdm import tqdm

# 모델 로드
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# 문장을 임베딩으로 변환하는 함수
def get_embeddings(sentences, batch_size=32):
    return model.encode(sentences, batch_size=batch_size, max_length=512)['dense_vecs']

# 컨텍스트 임베딩 생성
print("Generating context embeddings...")
context_embeddings = get_embeddings(df['context'].tolist())

# negative_samples 열을 추가할 리스트
negative_samples_list = []

# 각 question에 대해 유사도가 높은 4개의 다른 context를 찾기
print("Finding negative samples...")
for i in tqdm(range(len(df)), total=len(df), dynamic_ncols=True):
    question = df.iloc[i]['question']

    # 질문 임베딩 생성
    question_embedding = get_embeddings([question])[0]

    # 코사인 유사도 계산
    similarities = np.dot(context_embeddings, question_embedding)

    # 현재 문서의 인덱스를 제외한 4개의 문서를 선택
    top_n_indices = similarities.argsort()[::-1]
    top_n_indices = [idx for idx in top_n_indices if idx != i][:4]  # 현재 문서 제외, 상위 4개 선택

    # 선택된 문서들을 negative_samples 리스트로 추가
    negative_samples = [df['context'].iloc[idx] for idx in top_n_indices]
    negative_samples_list.append(negative_samples)

# 결과를 새로운 열로 추가
df['negative_samples'] = negative_samples_list

# 결과 확인
print(df.head())  # 결과의 처음 5개 행을 출력

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating context embeddings...


pre tokenize: 100%|█████████████████████████████████████████████████████████████████| 327/327 [00:01<00:00, 212.62it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Inference Embeddings: 100%|██████████████████████████████████████████████████████████| 327/327 [01:56<00:00,  2.80it/s]


Finding negative samples...


100%|████████████████████████████████████████████████████████████████████████████| 10434/10434 [07:57<00:00, 21.83it/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
2      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
3                                     나루세 요시히사          None  wikipedia   
4                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
8  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
2  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
3  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
4  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
8  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
2          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [19]:
df = df.reset_index(drop=True)

In [20]:
idx = 5

print('질문 :', df['question'].loc[idx])
print('정답 문서:', df['context'].loc[idx])
print('네거티브 문서:', df['negative_samples'].loc[idx])

질문 : 에티하드 웰니스 프로그램의 일환으로 위생에 관한 정보를 제공하는 것은 누구인가?
정답 문서: 아랍 에미리트의 국영항공사 에티하드항공이 지난 10월 21일 열린 ’2020비즈니스 트래블러 중동 어워드(Business Traveller Middle East Awards 2020)에서 ‘최고의 이코노미 클래스(Best Economy Class)’와 ‘최고의 상용고객 프로그램(Best Frequent Flyer Programme’)’상의 영예를 안았다. 이는 2012년과 2013년에 열린 비즈니스 트래블러 중동 어워드에서 각각 최고의 이코노미 클래스, 최고의 상용 고객프로그램 분야의 첫 수상을 한 이래로 2회째 수상 기록이다. 지난 해, 에티하드항공은 다양한 수상 경력을 지닌 비즈니스, 퍼스트, 더 레지던스 캐빈에서 펼쳐 온 혁신과 마찬가지로 이코노미석에서도 기내식 선택권을 넓히고 무선 기내 엔터테인먼트, 인체공학적 디자인으로 설계된 좌석과 여유로운 레그룸을 가진 이코노미석 도입을 비롯해 옆 좌석 비우기, 보다 넉넉한 이코노미 스페이스 석 선택 옵션, 선호좌석 선택 사항을 제공하는 등 기내 경험에 대한 더욱 다양한 선택권을 제공해 왔다. 이를 통해 승객이 개인의 필요와 요구에 맞는 맞춤화된 선택을 할 수 있도록 돕고 만족도 높은 항공 여행을 할 수 있도록 서비스와 상품 개선에 투자를 지속해 왔다. 또한, 에티하드항공은 항공사의 건강 및 위생 프로그램인 ‘에티하드 웰니스’프로그램을 통해 신종코로나바이러스로 인한 전례 없는 시기에 필수적인 여행을 해야만 하는 여행자들에게 마음의 평화와 여행에 대한 자신감을 심어주기 위해 펼친 다양한 노력이 고객의 긍정적인 평가를 받았다고 전했다. 예약 과정에서부터 공항 이용은 물론 항공여행에 이르기까지 인공지능 기술을 비롯한 최신 기술을 과감히 도입해 광범위한 예방 조치를 시행하고 있으며 업계 최초로 선보인 에티하드 웰니스 프로그램에서는 특별 훈련 과정을 거친 에티하드항공의 웰니스 엠버서더가 여행 전 과정에 걸친 조언과 건강 및

In [21]:
import datasets
from huggingface_hub import login
from datasets import load_dataset

In [ ]:
# API 토큰으로 로그인 (발급받은 토큰을 입력)
login("")

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-bge-m3")

In [22]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("iamjoon/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/77.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10434 [00:00<?, ? examples/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [23]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples
0,제주도 장마 시작 … 중부는 이달 말부터,종합,hankyung,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,1,False,한 달가량,478,[궤도물리학은 계절의 지속 기간이 지구의 궤도가 지점과 분점 사이의 공간을 휩쓸고 ...
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",경제,acrofan,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?,2,False,삼보테크놀로지,1422,"[IIoT(Industry Internet of Thing), 산업현장 스마트 팩토..."
2,나루세 요시히사,None,wikipedia,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,2,False,와쿠이 히데아키,107,[1960년에 주니치에 입단하여 같은 해 1960년 5월 7일 다이요 웨일스전에서 ...
3,편의점 휩쓴 ‘맛집 라면’,생활경제,hankyung,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,2,False,‘교동반점 짬뽕’,408,[‘짜파구리’로 시작된 ‘나만의 레시피’ 열풍이 올해 식품시장을 주도한 것으로 나타...
4,사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”,기획,hankyung,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,1,False,환경오염이 심한 지역,661,[포브스는 최근 “석탄은 기원전 315년 그리스 문헌에 대장간에서 원료로 쓰던 기록...
...,...,...,...,...,...,...,...,...,...,...
10429,인터뷰/ 이동기 신임 중견기업학회장,중기 의료,hankyung,“오너 경영은 중견기업이 글로벌 전문기업이나 대기업으로 성장하는 밑거름이 될 것입니...,이동기 학회장은 오너 경영을 무엇과 비교했는가?,1,False,전문 경영인 체제,229,[금융회사 지배구조 개선을 위해 은행과 자산총액 2조원 이상인 보험사와 증권사는 이...
10430,"엘리닉, 고객 감사의 달 맞아 팝업스토어 앵콜 오픈",산업,acrofan,홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?,1,False,롯데백화점 잠실점,134,[뉴욕관광청은 뉴욕이 가장 화려하게 빛나는 계절인 겨울에 뉴욕 여행을 계획하는 여행...
10431,유인원과 인간의 치열한 생존 전쟁,문화/TV,hankyung,유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?,2,False,‘혹성탈출: 반격의 서막’,170,[아오모리현의 핵연료 재처리 시설에서 의문의 두 사람에 의해 물체가 강탈된다. 강탈...
10432,"인텍앤컴퍼니, ASUS ROG STRIX GeForce RTX3080 OC 국내 출...",테크,acrofan,ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?,1,False,8시 10분,197,[법정관리(기업회생절차) 중인 팬택 매각의 본입찰 일정이 연기됐다. 인수의향서(LO...
